## Purpose of this works

<p> This Python script utilizes the BeautifulSoup library to perform web scraping on selected high street banks, including Barclays, NatWest, and Santander. The objective is to capture the latest interest rates for 1 Year Fixed Bonds from these banks' websites. <br> The script navigates through the HTML structure of each bank's webpage and extracts relevant information about the interest rates. To facilitate easy comparison, the script presents the obtained interest rates in descending order, allowing users to quickly identify and evaluate the most competitive offers. <br> This provides a convenient and automated way to stay updated on the current financial offerings from these prominent banks and make informed decisions based on the latest and most favorable rates. </p>

## Load in the necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re

## Perform web scraping on each bank's website

In [2]:
# Load the Barclays' webpage content
barclay_r = requests.get("https://www.barclays.co.uk/savings/bonds/1-year-fixed-rate-savings-bond/")

# Convert to a beautiful soup object
barclay = bs(barclay_r.content)

# Look for the index of AER% column
barclay_header = barclay.find_all('tr')[0]
barclay_p = barclay_header.find_all('p')
i = 0
for p in barclay_p:
    if p.get_text().startswith('AER'):
        break
    i+=1
      
# Get the rate from AER column
barclay_table = barclay.find_all('tr')[1]
barclay_one_year = barclay_table.find_all('p')[i]
rate_barclay = barclay_one_year.get_text(strip=True)
rate_barclay = float(rate_barclay.strip("%"))
rate_barclay

4.85

In [3]:
# Load the Natwest's webpage content
natwest_r = requests.get("https://www.natwest.com/savings/fixed-term-savings-account.html")

# Convert to a beautiful soup object
natwest = bs(natwest_r.content)

# Look for specific class name that contains information about 1 year term
natwest_term = natwest.find_all("span", string=re.compile("1 year"))[0]
natwest_class = natwest_term.parent['class']

# Point to the target table
natwest_table = natwest_term.parent.parent.parent

# Look for the index of AER% column
natwest_table_header = natwest_table.find_all(True, {'class': 'row0'})
j = 0
for header in natwest_table_header:
    if header.find_all("h4", string=re.compile("AER")):
        break
    j+=1

# Get the rate from AER column
natwest_one_year = natwest_table.find_all(True, {'class':natwest_class[0]})
rate_natwest = natwest_one_year[2].get_text(strip=True)
rate_natwest = float(rate_natwest.strip("%"))
rate_natwest

4.6

In [4]:
# Load the Santander's webpage content
santander_r = requests.get("https://www.santander.co.uk/personal/savings-and-investments/savings/fixed-term-bonds")

# Convert to a beautiful soup object
santander = bs(santander_r.content)

# Look for the table applies to new customers
santander_table = santander.find("table", id="tablefield-0")

# Look for the location of '1 Year' wording
santander_one_year = santander_table.find("p", string=re.compile("1 Year"))
# Look for the table contains information of 1 Year term
santander_one_year_table = santander_one_year.parent.parent
# Get the rate that has "AER" attribute
santander_aer = santander_one_year_table.find_all(attrs={"data-header": "AER/Gross"})[0]
rate_santander = santander_aer.get_text(strip=True)
rate_santander = float(rate_santander.strip("%"))
rate_santander


4.35

## Presents obtained interest rates in descending order

In [5]:
# Put all AER% results from web scraping into a hash map, i.e. dictionary
rates_dict = {"Barclays": rate_barclay, "Natwest": rate_natwest, "Santander": rate_santander}

sorted_rates = sorted(rates_dict.items(), key=lambda x:x[1], reverse=True)
sorted_rates

[('Barclays', 4.85), ('Natwest', 4.6), ('Santander', 4.35)]

In [6]:
# Show the sorted results

print("The interest rates for 1 Year Fixed Bond are as follows (in descending order):")

for rate in sorted_rates:
    print(f"Bank: {rate[0]}")
    print(f"Interest rate: {rate[1]}")

The interest rates for 1 Year Fixed Bond are as follows (in descending order):
Bank: Barclays
Interest rate: 4.85
Bank: Natwest
Interest rate: 4.6
Bank: Santander
Interest rate: 4.35
